In [17]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook

# Define the material properties and the strain gauge angles
E = 200e9  # Young's Modulus in Pascals (Pa)
v = 0.3  # Poisson's ratio
angles = [0, 45, 90]  # Strain gauge rosette angles in degrees

def transform_strains_to_global(epsilon_A, epsilon_B, epsilon_C, angles):
    """
    Transform strains from the rosette's local coordinate system to the global coordinate system.

    Parameters:
    - epsilon_A (float): Measured normal strain from the first strain gauge in the rosette.
    - epsilon_B (float): Measured normal strain from the second strain gauge in the rosette.
    - epsilon_C (float): Measured normal strain from the third strain gauge in the rosette.
    - angles (list of float): The angles (in degrees) of the strain gauges relative to the global x-axis.

    Returns:
    - numpy.array: The transformed strains in the global coordinate system, containing
      the normal strains epsilon_x and epsilon_y, along with the engineering shear strain gamma_xy.
    """
    theta_A, theta_B, theta_C = np.radians(angles)
    T = np.array([
        [np.cos(theta_A)**2, np.sin(theta_A)**2, 1 * np.sin(theta_A) * np.cos(theta_A)],
        [np.cos(theta_B)**2, np.sin(theta_B)**2, 1 * np.sin(theta_B) * np.cos(theta_B)],
        [np.cos(theta_C)**2, np.sin(theta_C)**2, 1 * np.sin(theta_C) * np.cos(theta_C)]
    ])
    T_inv = np.linalg.inv(T)
    local_strains = np.array([epsilon_A, epsilon_B, epsilon_C])
    global_strains = T_inv @ local_strains
    return global_strains

def calculate_principal_strains(epsilon_x, epsilon_y, gamma_xy):
    """
    Calculate the principal strains using Mohr's circle relations.

    Parameters:
    - epsilon_x (float): Normal strain in the x-direction (global coordinate system).
    - epsilon_y (float): Normal strain in the y-direction (global coordinate system).
    - gamma_xy (float): Engineering shear strain in the global coordinate system.

    Returns:
    - numpy.array: An array containing the maximum (epsilon_1) and minimum (epsilon_2)
      principal strains. These values are critical for assessing the material's behavior
      under stress and for failure analysis.

    Note:
    - The shear strain input should be in engineering terms (total angular deformation),
      as this is the standard output from strain gauges.
    """
    C = (epsilon_x + epsilon_y) / 2
    R = np.sqrt(((epsilon_x - epsilon_y) / 2)**2 + (gamma_xy / 2)**2)
    epsilon_1 = C + R  # Maximum principal strain
    epsilon_2 = C - R  # Minimum principal strain
    return np.array([epsilon_1, epsilon_2])

def calculate_principal_stresses(principal_strains, E, v):
    """
    Calculate the principal stresses from the principal strains using material properties.

    Given the principal strains, this function applies Hooke's law in two dimensions to
    compute the principal stresses. The material's Young's modulus (E) and Poisson's ratio (v)
    are used to relate the strains to the stresses. This function assumes a linear elastic
    material behavior and plane stress conditions, which is a common scenario in thin structures
    where one dimension is significantly smaller than the other two.

    Parameters:
    - principal_strains (numpy.array): An array of the principal strains [epsilon_1, epsilon_2].
    - E (float): Young's Modulus of the material in Pascals (Pa).
    - v (float): Poisson's ratio of the material, dimensionless.

    Returns:
    - numpy.array: An array containing the principal stresses [sigma_1, sigma_2] in Pascals (Pa).
      These stresses are the maximum and minimum normal stresses that occur at the principal
      strain orientations, where the shear stress is zero.

    Note:
    - The principal strains should be provided in microstrains (με) for the calculation.
    - The function returns the principal stresses in Pascals (Pa), but they are manually
      converted to other MPa (MegaPascals) at the function return (by a /1e6 division).
    - The function holds true for isotropic materials where the stress-strain relationship 
      is governed by the isotropic form of Hooke's law.
    """
    #S = np.array([
    #    [1, v, 0],
    #    [v, 1, 0],
    #    [0, 0, (1-v)/2]
    #]) * E / (1 - v**2)
    S = np.array([
        [1, v],
        [v, 1]
    ]) * E / (1 - v**2)

    principal_stresses = S @ (principal_strains /1e6)
    return principal_stresses / 1e6  # Convert Pa to MPa

def calculate_principal_strain_orientation(epsilon_x, epsilon_y, gamma_xy):
    """
    Calculate the orientation of the principal strains from the original strain measurements using Mohr's circle analysis.
    Parameters:
    - epsilon_x (float): Normal strain in the x-direction.
    - epsilon_y (float): Normal strain in the y-direction.
    - gamma_xy (float): Engineering shear strain.
    Returns:
    - theta_p (float): Angle of the principal strain in degrees.
    """
    # Calculate the angle to the maximum principal strain
    theta_p_rad = 0.5 * np.arctan2(gamma_xy, epsilon_x - epsilon_y)
    theta_p = np.degrees(theta_p_rad)

    # Adjust the angle to ensure it's within the 0-180 degree range
    if theta_p < 0:
        theta_p += 180

    return theta_p

def calculate_biaxiality_ratio(S1, S2):
    """
    Calculate the biaxiality ratio from the principal stresses.

    The biaxiality ratio is the ratio of the second principal stress to the first principal stress.
    It is a dimensionless number that indicates the relation between the principal stresses. A
    biaxiality ratio of 1 indicates equal biaxial stress state, while a value of 0 indicates a uniaxial
    stress state. Negative values indicate that the principal stresses are of opposite signs.

    Parameters:
    - S1 (float or numpy.array): First principal stress (assumed to be the larger one in absolute value).
    - S2 (float or numpy.array): Second principal stress (assumed to be the smaller one in absolute value).

    Returns:
    - float or numpy.array: The biaxiality ratio, a dimensionless quantity indicating the biaxial state of stress.

    Note:
    - The function assumes that S1 and S2 are provided such that the absolute value of S1 is
      greater than or equal to the absolute value of S2. If this is not the case, the inputs
      should be swapped.
    """
    # Ensure that sigma_1 is the larger one in absolute terms
    sigma_1 = np.where(np.abs(S1) >= np.abs(S2), S1, S2)
    sigma_2 = np.where(np.abs(S1) >= np.abs(S2), S2, S1)
    
    # Calculate the biaxiality ratio
    biaxiality_ratio = sigma_2 / sigma_1
    
    return biaxiality_ratio

def calculate_von_mises_stress(S1, S2, S3=0):
    """
    Calculate the von Mises stress from the principal stresses.
    
    Parameters:
    S1 (float): First principal stress in MPa.
    S2 (float): Second principal stress in MPa.
    S3 (float): Third principal stress in MPa, assumed to be zero for plane stress condition.
    
    Returns:
    float: The von Mises stress in MPa.
    """
    # Calculate the von Mises stress using the principal stresses
    sigma_vm = np.sqrt(((S1 - S2)**2 + (S1 - S3)**2 + (S2 - S3)**2) / 2)
    return sigma_vm


# Load the CSV file into a pandas DataFrame
file_path = 'SG_sample_out_data_sg_filtered_v2_long_test.csv'  # Path to the provided file
data = pd.read_csv(file_path)  # Read the entire data

# Save 'Data No' and 'Time' columns for later
data_number = data['Data No']
time = data['Time']

# Filter only the strain gauge data, skipping the first two columns
strain_gauge_data = data.iloc[:, 2:].filter(regex='SG')

# Calculate the number of rosettes (each rosette has 3 strain measurements)
num_rosettes = (len(strain_gauge_data.columns)) // 3

# Process each set of strain gauges for each rosette
for i in range(num_rosettes):
    
    rosette_id = i + 1

    strain_gauge_cols = [f'SG{i+1}_1', f'SG{i+1}_2', f'SG{i+1}_3']
    strains = strain_gauge_data[strain_gauge_cols].values  # in microstrains

    # Transform strains to the global coordinate system
    global_strains = np.array([transform_strains_to_global(*strain, angles) for strain in strains])

    # Calculate principal strains from the global strains using Mohr's circle
    principal_strains = np.array([calculate_principal_strains(strain[0], strain[1], strain[2]) for strain in global_strains])  # in microstrains

    principal_stresses = np.array([calculate_principal_stresses(strain, E, v) for strain in principal_strains])  # principal strains are already in strains
    principal_strain_orientation = np.array([calculate_principal_strain_orientation(strain[0], strain[1], strain[2]) for strain in global_strains])
    biaxiality_ratios = calculate_biaxiality_ratio(principal_stresses[:, 0], principal_stresses[:, 1])
    von_mises_stresses = np.array([calculate_von_mises_stress(*stress) for stress in principal_stresses])

    # Add the calculated data to the DataFrame using the correct naming convention
    strain_gauge_data[f'SG{i+1}_epsilon_x[με]'], strain_gauge_data[f'SG{i+1}_epsilon_y[με]'], strain_gauge_data[f'SG{i+1}_gamma_xy[με]'] = global_strains.T
    strain_gauge_data[f'SG{i+1}_sigma_1[MPa]'], strain_gauge_data[f'SG{i+1}_sigma_2[MPa]'] = principal_stresses.T
    strain_gauge_data[f'SG{i+1}_theta_p[°]'] = principal_strain_orientation
    strain_gauge_data[f'SG{i+1}_Biaxiality_Ratio'] = biaxiality_ratios
    strain_gauge_data[f'SG{i+1}_von_Mises[MPa]'] = von_mises_stresses


# Add the 'Data No' and 'Time' columns back to the DataFrame
strain_gauge_data.insert(0, 'Time', time)
strain_gauge_data.insert(0, 'Data No', data_number)

# Display the DataFrame with the new calculated columns to verify
print(strain_gauge_data.head())

excel_file_path = "strain_gauge_data_results.xlsx"
strain_gauge_data.to_excel(excel_file_path)

# Load the workbook and select the active worksheet
wb = load_workbook(excel_file_path)
ws = wb.active

# Determine the columns to hide
# Assuming 'Time' is in the first column and 'von Mises' stress columns are identified by a pattern
columns_to_hide = [col for col in ws.columns if col[0].value is not None 
                   and 'von_Mises' not in col[0].value 
                   and col[0].value != 'Time']

# Hide these columns
for column in columns_to_hide:
    ws.column_dimensions[column[0].column_letter].hidden = True

wb.save(excel_file_path)

   Data No    Time      SG1_1     SG1_2     SG1_3      SG2_1     SG2_2  \
0        1  0.0002   0.176820  0.079159 -0.039136   0.176820  0.079159   
1        2  0.0004   0.033286  0.079361  0.034511   0.033286  0.079361   
2        3  0.0006  -2.079800 -0.607560  0.676880  -2.079800 -0.607560   
3        4  0.0008  -6.545800 -2.144700  1.997400  -6.545800 -2.144700   
4        5  0.0010 -12.295000 -4.146900  3.711600 -12.295000 -4.146900   

      SG2_3  SG1_epsilon_x[με]  SG1_epsilon_y[με]  ...  SG1_Biaxiality_Ratio  \
0 -0.039136           0.176820          -0.039136  ...              0.082006   
1  0.034511           0.033286           0.034511  ...              0.161295   
2  0.676880          -2.079800           0.676880  ...             -0.029365   
3  1.997400          -6.545800           1.997400  ...             -0.005890   
4  3.711600         -12.295000           3.711600  ...             -0.002148   

   SG1_von_Mises[MPa]  SG2_epsilon_x[με]  SG2_epsilon_y[με]  SG2_gamma_xy[

SONUCLARI KONTROL ET!!!

In [18]:
for column in ws.columns:
        print(f"Header: {column[0].value}, Column: {column[0].column_letter}")

Header: None, Column: A
Header: Data No, Column: B
Header: Time, Column: C
Header: SG1_1, Column: D
Header: SG1_2, Column: E
Header: SG1_3, Column: F
Header: SG2_1, Column: G
Header: SG2_2, Column: H
Header: SG2_3, Column: I
Header: SG1_epsilon_x[με], Column: J
Header: SG1_epsilon_y[με], Column: K
Header: SG1_gamma_xy[με], Column: L
Header: SG1_sigma_1[MPa], Column: M
Header: SG1_sigma_2[MPa], Column: N
Header: SG1_theta_p[°], Column: O
Header: SG1_Biaxiality_Ratio, Column: P
Header: SG1_von_Mises[MPa], Column: Q
Header: SG2_epsilon_x[με], Column: R
Header: SG2_epsilon_y[με], Column: S
Header: SG2_gamma_xy[με], Column: T
Header: SG2_sigma_1[MPa], Column: U
Header: SG2_sigma_2[MPa], Column: V
Header: SG2_theta_p[°], Column: W
Header: SG2_Biaxiality_Ratio, Column: X
Header: SG2_von_Mises[MPa], Column: Y
